# Runtimes Cookbook

## Overview

In this tutorial, we will explore the `Runtimes` in CAMEL, focusing on three specific runtime implementations: 

1. `LLMGuardRuntime`: This runtime evaluates the risk level of functions using a language model. It ensures that potentially harmful functions are assessed and controlled before execution. The runtime uses a prompt-based approach to determine the risk score of a function based on its description and parameters.

2. `RemoteHttpRuntime`: This runtime allows functions to be executed on a remote HTTP server. It is useful for distributing workloads across different servers and ensuring that functions run in a controlled remote environment. The runtime supports custom entry points and arguments, and provides mechanisms to check server status and wait for readiness.

3. `DockerRuntime`: This runtime provides a containerized environment for executing functions using Docker. It ensures that functions run in isolated and reproducible environments, making it ideal for complex workflows and dependency management. The runtime supports mounting directories, copying files to containers, and adding tasks to execute commands inside containers.

Each of these runtimes provides a unique environment for executing functions, ensuring safety, remote execution, and containerized execution, respectively.

## Introduction



The `BaseRuntime` class is the base class for runtime environments used in the CAMEL system. It is designed to provide a consistent structure for executing functions in different environments and allows for easy extension to support various runtime implementations. The `BaseRuntime` class ensures that functions are executed in a controlled and predictable manner, providing mechanisms for initialization, execution, and cleanup.

### Installation

Ensure you have CAMEL AI installed in your Python environment:

In [ ]:
!pip install "camel-ai==0.2.14"

If you want to try DockerRuntime, please make sure you have Docker installed in your environment.

### Setting Up API Keys

You'll need to set up your API keys for OpenAI.

In [1]:
import os
from getpass import getpass

# Prompt for the API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key


In [ ]:
import logging

class RuntimeLogFilter(logging.Filter):
    def filter(self, record):
        return "camel.runtimes" in record.name


logger = logging.getLogger()
logger.setLevel(logging.INFO)

stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
stream_handler.addFilter(RuntimeLogFilter())


formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
stream_handler.setFormatter(formatter)

logger.addHandler(stream_handler)


## LLMGuardRuntime

### Overview

`LLMGuardRuntime` is a specialized runtime designed to evaluate and manage the risks associated with executing functions in applications where safety, compliance, or security are critical. It leverages a language model to assess potential risks and provides tools to mitigate or ignore them based on predefined criteria.


### Key Features

- **Risk Evaluation**: Utilizes a language model to evaluate the risk associated with executing a function.
- **Risk Scoring**: Assigns scores to functions based on predefined risk criteria.
- **Risk Mitigation**: Allows for the implementation of mechanisms to ignore or handle certain risks, offering flexibility in application behavior.
- **Seamless Integration**: Compatible with tools in camel for easy adoption and integration into existing workflows.


In [4]:
from camel.runtimes.llm_guard_runtime import LLMGuardRuntime
from camel.toolkits import FunctionTool


def example_function(a, b):
    return a + b


tool = FunctionTool(example_function)
runtime = LLMGuardRuntime()
runtime.add(tool)
result = tool.func(1, 2)
print(result)

2024-12-26 22:57:57,429 - camel.agents.chat_agent - WARNING - Overriding the configured tools in `BaseModelBackend` with the tools from `ChatAgent`.
2024-12-26 22:57:58,500 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-26 22:57:58,502 - camel.agents.chat_agent - INFO - Model gpt-4o-mini, index 0, processed these messages: [{'role': 'system', 'content': "You are a function safety evaluator tasked with assessing the \npotential risk level of a given function based on both its description \nand parameters. Your goal is to determine if the function may pose any \nharm to the user's environment, such as deleting files, executing \narbitrary code, or accessing sensitive information. Pay special attention \nto the provided parameters — even if a function has the potential to be \ndangerous, its actual parameters may indicate harmless behavior, and the \nrisk level should be adjusted accordingly. Use the `function_risk` tool to \nassig

2024-12-26 22:57:58,503 - camel.runtimes.llm_guard_runtime - INFO - Function example_function passed risk assessment.Score: 1, Reason: The function 'example_function' takes two integer arguments and has no keyword arguments. Since it appears to perform a simple operation with these integers, such as a mathematical calculation, it poses no harm to the user's environment.


3



### When to Use `LLMGuardRuntime`

You should consider using `LLMGuardRuntime` in the following scenarios:

1. **Sensitive Operations**: When the execution of functions may involve sensitive or critical data, and an evaluation of risks is essential.
   
2. **Compliance Requirements**: In environments where regulatory or compliance frameworks demand thorough risk assessments before executing operations.

3. **Dynamic Risk Management**: When applications require dynamic evaluation and scoring of functions to determine their safety and alignment with predefined criteria.

4. **Controlled Flexibility**: If you need the ability to selectively ignore certain risks while maintaining control over others.

5. **AI-Driven Decision Making**: When leveraging AI to predict and mitigate risks associated with runtime operations can add value.



### Advantages

- **Proactive Risk Mitigation**: By evaluating functions before execution, potential issues can be flagged and addressed preemptively.
- **Customizable Criteria**: Users can define their own risk criteria, tailoring the runtime to specific application needs.
- **Enhanced Security**: Reduces the likelihood of unintended consequences by monitoring and managing function execution risks.

## RemoteHttpRuntime

### Overview

`RemoteHttpRuntime` is a runtime designed to execute functions on a remote HTTP server, enabling distributed and scalable execution of tasks. It supports customization of entry points, arguments, and provides mechanisms to ensure server readiness and reliability during execution.

### Key Features

- **Remote Execution**: Executes functions on a designated HTTP server for distributed processing.
- **Custom Entry Points**: Allows configuration of custom entry points and arguments for tailored operations.
- **Seamless Integration**: Compatible with tools in camel for easy adoption and integration into existing workflows.

In [3]:
from camel.runtimes.remote_http_runtime import RemoteHttpRuntime
from camel.toolkits.math_toolkit import MathToolkit

runtime = (
        RemoteHttpRuntime("localhost")
        .add(MathToolkit().get_tools(), "camel.toolkits.MathToolkit")
        .build()
    )
print("Waiting for runtime to be ready...")
runtime.wait()
print("Runtime is ready.")
add, sub, mul, _, _ = runtime.get_tools()
print(f"Add 1 + 2: {add.func(1, 2)}")
print(f"Subtract 5 - 3: {sub.func(5, 3)}")
print(f"Multiply 2 * 3: {mul.func(2, 3)}")

print("Documents: ", runtime.docs)
# you can open this url in browser to see the API Endpoints
# before the runtime is stopped.
# time.sleep(60)
runtime.stop()
# call runtime.stop() if you want to stop the runtime manually
# atherwise it will be stopped automatically when the program ends

Waiting for runtime to be ready...
2024-12-12 23:30:51,108 - camel - INFO - Camel library logging has been configured.
2024-12-12 23:30:51,728 - __main__ - INFO - Modules and functions: ['camel.toolkits.MathToolkit']
2024-12-12 23:30:51,729 - __main__ - INFO - Importing camel.toolkits and function MathToolkit


INFO:     Will watch for changes in these directories: ['/home/lxk/camel/docs/cookbooks']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [674247] using WatchFiles


2024-12-12 23:30:52,167 - camel - INFO - Camel library logging has been configured.
2024-12-12 23:30:52,819 - __mp_main__ - INFO - Modules and functions: ['camel.toolkits.MathToolkit']
2024-12-12 23:30:52,820 - __mp_main__ - INFO - Importing camel.toolkits and function MathToolkit


INFO:     Started server process [674287]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


INFO:     127.0.0.1:60696 - "GET / HTTP/1.1" 404 Not Found
Runtime is ready.
INFO:     127.0.0.1:60410 - "POST /add HTTP/1.1" 200 OK
Add 1 + 2: 3
INFO:     127.0.0.1:60420 - "POST /sub HTTP/1.1" 200 OK
Subtract 5 - 3: 2
INFO:     127.0.0.1:60436 - "POST /multiply HTTP/1.1" 200 OK
Multiply 2 * 3: 6
Documents:  http://localhost:8000/docs


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [674287]
INFO:     Stopping reloader process [674247]


### When to Use `RemoteHttpRuntime`

Consider using `RemoteHttpRuntime` in the following scenarios:

1. **Distributed Computing**: When tasks need to be executed across remote servers for scalability and performance.
   
2. **Centralized Function Hosting**: If functions are hosted on a central server and accessed remotely by various clients.

3. **Customizable API Endpoints**: For applications requiring custom entry points and specific arguments for remote execution.


### Advantages

- **Scalability**: Enables distributed task execution across remote servers, enhancing scalability.
- **Flexibility**: Supports custom configurations to meet specific application needs.
- **Reliability**: Provides robust mechanisms to verify runtime readiness before execution.
- **Integration-Friendly**: Compatible with toolkits for quick and efficient integration.



### Notes on Usage

1. **Server Address**: Ensure the correct HTTP server address is provided during initialization.
2. **Documentation Access**: The `runtime.docs` property provides API documentation for easy reference.
3. **Runtime Stopping**: Always stop the runtime using `runtime.stop()` to free resources when it is no longer needed.

## DockerRuntime

### Overview

`DockerRuntime` enables the execution of functions within Docker containers, providing a controlled and isolated environment for running tasks. It supports directory mounting, file transfers, and seamless task execution, making it ideal for use cases requiring reproducibility and isolation.


### Key Features

- **Docker-Based Execution**: Executes functions within Docker containers, ensuring an isolated runtime environment.
- **Directory Mounting**: Allows mounting of local directories into the container for resource sharing.
- **File Transfer Support**: Facilitates copying files to and from the container for streamlined workflows.
- **Task Execution**: Provides mechanisms to add tasks and execute commands inside the container.
- **Seamless Integration**: Compatible with tools in camel for easy adoption and integration into existing workflows.

In [ ]:
from camel.runtimes import DockerRuntime
from camel.toolkits.code_execution import CodeExecutionToolkit


# tools
toolkit = CodeExecutionToolkit(verbose=True)

# change to your own docker image
runtime = DockerRuntime("xukunliu/camel").add(
    toolkit.get_tools(),
    "camel.toolkits.CodeExecutionToolkit",
    redirect_stdout=True,
    arguments=dict(verbose=True),
)

tool = runtime.get_tools()[0]

with runtime as r:
    print("Waiting for runtime to be ready...")
    r.wait()
    print("Runtime is ready.")
    tool.func("2 ** 10")

print("DockerRuntime stopped")
    

2024-12-12 23:34:31,918 - camel.runtimes.docker_runtime - INFO - Copying /home/lxk/camel/camel/runtimes/api.py to /home
2024-12-12 23:34:31,983 - camel.runtimes.docker_runtime - INFO - Container started on port 8000
Waiting for runtime to be ready...
Runtime is ready.
Executed the code below:
```py
2 ** 10
```
> Executed Results:
1024

2024-12-12 23:34:46,676 - camel.runtimes.docker_runtime - INFO - Removing container.



### When to Use `DockerRuntime`

You should consider using `DockerRuntime` in the following scenarios:

1. **Environment Isolation**: When tasks require isolated environments to avoid interference or conflicts with the host system.
   
2. **Reproducibility**: For workflows that demand reproducible environments across different systems.

3. **Resource Sharing**: If tasks involve sharing resources such as files or directories between the host and the container.

4. **Code Execution**: When executing scripts or commands using a toolkit like `CodeExecutionToolkit` within a Dockerized environment.

5. **Customized Docker Images**: If your application depends on specific Docker images configured with the required dependencies.



### Advantages

- **Isolation**: Ensures tasks are executed in a clean and controlled environment.
- **Flexibility**: Allows integration with toolkits for custom workflows.
- **Resource Sharing**: Facilitates efficient sharing of data and files with containers.
- **Ease of Use**: Provides a high-level API for managing and interacting with Docker containers.


### Notes on Usage

1. **Docker Image**: Replace `"xukunliu/camel"` with your specific Docker image.
2. **Directory Mounting**: Use directory mounting to provide access to local files inside the container if needed.
3. **File Management**: Leverage file transfer capabilities to streamline workflows.
4. **Stopping Runtime**: The runtime is automatically stopped when the program ends or when exiting the context manager.


## Use `Runtimes` with Agent

In this section, we will demonstrate how to use the `Runtime` with an agent in the CAMEL system. 

The `Runtime` class provides an `add` function that allows you to add `FunctionTool` instances to the runtime and use `get_tools` to retrieve the wrapped functions. These functions behave exactly like the original functions, and you can add them to the agent just like any other tools.

In [9]:
from colorama import Fore
from camel.agents import ChatAgent
from camel.configs import ChatGPTConfig
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.runtimes import DockerRuntime
from camel.toolkits.code_execution import CodeExecutionToolkit
from camel.types import ModelPlatformType, ModelType
from camel.utils import print_text_animated

# Initialize the CodeExecutionToolkit
toolkit = CodeExecutionToolkit(verbose=True)

# Initialize the DockerRuntime with a specified Docker image
runtime = DockerRuntime("xukunliu/camel").add(
    toolkit.get_tools(),
    "camel.toolkits.CodeExecutionToolkit",
    redirect_stdout=True,
)

# Retrieve the tools from the runtime
tools = runtime.get_tools()

# Configure the language model
assistant_model_config = ChatGPTConfig(
    temperature=0.0,
)

# Create the model using the ModelFactory
model = ModelFactory.create(
    model_platform=ModelPlatformType.DEFAULT,
    model_type=ModelType.GPT_4O,
    model_config_dict=assistant_model_config.as_dict(),
)

# Create the system message for the agent
assistant_sys_msg = BaseMessage.make_assistant_message(
    role_name="Teacher",
    content=(
        "You are a personal math tutor and programmer. "
        "When asked a math question, "
        "write and run Python code to answer the question."
    ),
)

# Initialize the ChatAgent with the system message, model, and tools
agent = ChatAgent(
    assistant_sys_msg,
    model,
    tools=tools,
)
agent.reset()

# Use the runtime to execute the task
with runtime as r:
    r.wait()
    prompt = (
        "Weng earns $12 an hour for babysitting. "
        "Yesterday, she just did 51 minutes of babysitting. How much did she earn?"
    )
    user_msg = BaseMessage.make_user_message(role_name="User", content=prompt)
    print(Fore.YELLOW + f"user prompt:\n{prompt}\n")

    # Get the agent's response to the user message
    response = agent.step(user_msg)
    for msg in response.msgs:
        print_text_animated(Fore.GREEN + f"Agent response:\n{msg.content}\n")

2024-12-12 23:54:28,291 - camel.agents.chat_agent - WARNING - Overriding the configured tools in `BaseModelBackend` with the tools from `ChatAgent`.
2024-12-12 23:54:30,513 - camel.runtimes.docker_runtime - INFO - Copying /home/lxk/camel/camel/runtimes/api.py to /home
2024-12-12 23:54:30,605 - camel.runtimes.docker_runtime - INFO - Container started on port 8000
user prompt:
Weng earns $12 an hour for babysitting. Yesterday, she just did 51 minutes of babysitting. How much did she earn?

2024-12-12 23:54:38,240 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-12 23:54:38,304 - camel.agents.chat_agent - INFO - Model gpt-4o, index 0, processed these messages: [{'role': 'system', 'content': 'You are a personal math tutor and programmer. When asked a math question, write and run Python code to answer the question.'}, {'role': 'user', 'content': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 51 minutes of babysitting.

In this session, we introduced the BaseRuntime class and its specific runtime implementations. These components play essential roles in the CAMEL system, facilitating the execution of functions in various environments with safety, remote execution, and containerized execution.